In [1]:
import polars as pl

df = pl.read_csv("中国专利转让数据库1998-2024年.csv")
print(df.shape)
df = df.filter(pl.col("申请日") >= "2010-01-01")
print(df.shape)

# 只保留选中的列
df = df.select([
    "申请号",
    "历史法律状态"
])
print(f"筛选列后的形状: {df.shape}")


(4275769, 40)
(3675330, 40)
筛选列后的形状: (3675330, 2)


In [2]:
import polars as pl
import time

def expand_legal_status_native(df: pl.DataFrame) -> pl.DataFrame:
    """
    使用Polars原生操作的高性能版本
    """
    print(f"处理 {len(df):,} 行数据...")
    start_time = time.time()
    
    # 使用Polars原生字符串操作和向量化处理
    result = (
        df.with_row_index()
        # 按#分割法律状态记录
        .with_columns(
            pl.col("历史法律状态")
            .str.split("#")
            .alias("status_blocks")
        )
        # 展开每个状态块
        .explode("status_blocks")
        # 过滤空值
        .filter(pl.col("status_blocks").is_not_null() & (pl.col("status_blocks").str.len_chars() > 0))
        # 提取法律状态公告日
        .with_columns([
            pl.col("status_blocks")
            .str.extract(r"法律状态公告日：([^；\n]+)")
            .alias("法律状态公告日"),
            
            # 提取法律状态
            pl.col("status_blocks")
            .str.extract(r"法律状态：([^；\n]+)")
            .alias("当前法律状态"),
            
            # 提取描述信息
            pl.col("status_blocks")
            .str.extract(r"描述信息：([^；\n]+)")
            .alias("状态描述信息"),
            
            # 保留原始数据
            pl.col("历史法律状态").alias("原始历史法律状态")
        ])
        # 重新排列列顺序
        .select([
            "法律状态公告日",
            "当前法律状态", 
            "状态描述信息",
            "申请号",
            "原始历史法律状态"
        ])
    )
    
    total_time = time.time() - start_time
    print(f"处理完成！原始记录: {len(df):,}, 扩展后记录: {len(result):,}")
    print(f"总耗时: {total_time:.2f} 秒")
    
    return result

def expand_legal_status_alternative(df: pl.DataFrame) -> pl.DataFrame:
    """
    替代方案：使用map_elements但批量处理
    """
    print(f"处理 {len(df):,} 行数据...")
    start_time = time.time()
    
    def parse_status_batch(status_text):
        """批量解析单个状态字符串"""
        if not status_text or status_text == "":
            return [{"法律状态公告日": None, "当前法律状态": None, "状态描述信息": None}]
        
        blocks = [block.strip() for block in str(status_text).split('#') if block.strip()]
        if not blocks:
            return [{"法律状态公告日": None, "当前法律状态": None, "状态描述信息": None}]
        
        results = []
        for block in blocks:
            result = {"法律状态公告日": None, "当前法律状态": None, "状态描述信息": None}
            
            lines = block.split('\n')
            for line in lines:
                line = line.strip()
                if '法律状态公告日：' in line:
                    result["法律状态公告日"] = line.split('法律状态公告日：', 1)[1].split('；')[0].strip()
                elif '法律状态：' in line:
                    result["当前法律状态"] = line.split('法律状态：', 1)[1].split('；')[0].strip()
                elif '描述信息：' in line:
                    result["状态描述信息"] = line.split('描述信息：', 1)[1].split('；')[0].strip()
            
            results.append(result)
        
        return results
    
    # 使用map_elements进行向量化处理
    result = (
        df.with_row_index()
        .with_columns(
            pl.col("历史法律状态")
            .map_elements(parse_status_batch, return_dtype=pl.List(pl.Struct({
                "法律状态公告日": pl.Utf8,
                "当前法律状态": pl.Utf8, 
                "状态描述信息": pl.Utf8
            })))
            .alias("parsed_status")
        )
        .explode("parsed_status")
        .unnest("parsed_status")
        .with_columns(pl.col("历史法律状态").alias("原始历史法律状态"))
        .select([
            "法律状态公告日",
            "当前法律状态",
            "状态描述信息", 
            "申请号",
            "原始历史法律状态"
        ])
    )
    
    total_time = time.time() - start_time
    print(f"处理完成！原始记录: {len(df):,}, 扩展后记录: {len(result):,}")
    print(f"总耗时: {total_time:.2f} 秒")
    
    return result

print("已加载Polars原生优化版本")

已加载Polars原生优化版本


In [ ]:
expanded_df = expand_legal_status_native(df)
# 使用Polars原生操作版本（推荐）


# 如果上面的版本有问题，可以尝试替代方案
# expanded_df = expand_legal_status_alternative(df)

expanded_df = expanded_df.unique()
expanded_df.write_parquet("expanded_df.parquet")

处理 3,675,330 行数据...
处理完成！原始记录: 3,675,330, 扩展后记录: 14,795,686
总耗时: 24.36 秒
